In [22]:
import random
import pandas as pd
import os

In [23]:
def has_x_path(board):
    start_positions = [(0, c) for c in range(board_size) if board[0][c] == 'X']
    visited = set()
    stack = start_positions
    while stack:
        r, c = stack.pop()
        if (r, c) in visited:
            continue
        visited.add((r, c))
        if r == board_size - 1:
            return True
        for nr, nc in neighbors(r, c, board_size):
            if board[nr][nc] == 'X' and (nr, nc) not in visited:
                stack.append((nr, nc))
    return False

In [24]:
def has_o_path(board):
    start_positions = [(r, 0) for r in range(board_size) if board[r][0] == 'O']
    visited = set()
    stack = start_positions
    while stack:
        r, c = stack.pop()
        if (r, c) in visited:
            continue
        visited.add((r, c))
        if c == board_size - 1:
            return True
        for nr, nc in neighbors(r, c, board_size):
            if board[nr][nc] == 'O' and (nr, nc) not in visited:
                stack.append((nr, nc))
    return False

In [25]:
def board_to_string(board):
    return ''.join(''.join(row) for row in board)

In [26]:
def simulate_game(board_size):
    board = [['E' for _ in range(board_size)] for _ in range(board_size)]
    current_player = random.choice(['X', 'O'])
    empty_cells = [(r, c) for r in range(board_size) for c in range(board_size)]

    while empty_cells:
        r, c = random.choice(empty_cells)
        empty_cells.remove((r, c))
        board[r][c] = current_player

        if current_player == 'X' and has_x_path(board):
            return board, 0  # X winner
        elif current_player == 'O' and has_o_path(board):
            return board, 1  # O winner

        current_player = 'O' if current_player == 'X' else 'X'

    return None, None

In [27]:
def neighbors(r, c, size):
    # Hex-like neighbors (6 directions in a hex grid):
    # Adjust according to your hex definition.
    # For standard "hex" connectivity, consider the following:
    # On a hex board represented in a 2D array,
    # neighbors could be something like (r-1,c), (r+1,c), (r,c-1), (r,c+1), (r-1,c+1), (r+1,c-1)
    # Adjusted here for a hex-shaped connection on a rectangular board.
    directions = [
        (-1, 0),
        (1, 0),
        (0, -1),
        (0, 1),
        (-1, 1),
        (1, -1)
    ]
    for dr, dc in directions:
        nr, nc = r + dr, c + dc
        if 0 <= nr < size and 0 <= nc < size:
            yield nr, nc


In [28]:
# Parameters
board_size = 4
num_samples = 2000
desired_per_class = num_samples // 2  # 1000 X winners and 1000 O winners

rows = []
count_x = 0
count_o = 0

if os.path.exists("hex_data.csv"):
    os.remove("hex_data.csv")
    print("Removed hex_data.csv")

while count_x < desired_per_class or count_o < desired_per_class:
    final_board, label = simulate_game(board_size)
    if final_board is not None:
        if label == 0 and count_x < desired_per_class:  # X winner
            board_str = board_to_string(final_board)
            rows.append((board_str, label))
            count_x += 1
        elif label == 1 and count_o < desired_per_class:  # O winner
            board_str = board_to_string(final_board)
            rows.append((board_str, label))
            count_o += 1

    # Print progress every 100 samples collected
    total_collected = count_x + count_o
    if total_collected % 100 == 0:
        print(f"Collected {total_collected} / {num_samples} samples.")

df = pd.DataFrame(rows)
df.to_csv("hex_data.csv", index=False)
print(f"Generated {board_size}x{board_size} balanced dataset ({count_x} X wins and {count_o} O wins) saved as hex_data.csv")

Removed hex_data.csv
Collected 100 / 1000 samples.
Collected 200 / 1000 samples.
Collected 300 / 1000 samples.
Collected 400 / 1000 samples.
Collected 500 / 1000 samples.
Collected 600 / 1000 samples.
Collected 700 / 1000 samples.
Collected 800 / 1000 samples.
Collected 900 / 1000 samples.
Collected 1000 / 1000 samples.
Generated 4x4 balanced dataset (500 X wins and 500 O wins) saved as hex_data.csv
